In [5]:
from pathlib import Path
from paraphone.tasks.synth import GoogleSpeakSynthesizer, VOICES
from paraphone.tasks.phonemize import PhonemizedWordsCSV

synth = GoogleSpeakSynthesizer("fr", VOICES["fr"][0],
                               Path("../morphologicalstudy-1224861510d4.json"))

words = {"jeune", "jeuner", "os", "osseux", "oeuf", "euphémisme", "bonne",
         "beaune", "tomme", "tome", "taume", "comme", "côme", "gomme",
         "somme", "chaume", "nonne", "môme", "bol", "tôle", "col", "folle",
         "vol", "geole", "molle", "grolle"}
pho_word_csv = PhonemizedWordsCSV(Path("../workspaces/large_fr/phonemized/all.csv"))
pho_dict = pho_word_csv.to_dict()
tested_words = {
    word: " ".join(pho_dict[word]) for word in words if word in pho_dict
}
# TODO:

In [6]:
ssml_template = """
<speak>
{word}
<break time="1s"/>
<phoneme alphabet="ipa" ph="{phonemes}"></phoneme>
</speak>
"""

In [7]:
OUTPUT_DIR = Path("/tmp/synth_tests/")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
from asyncio import get_event_loop, ensure_future
loop = get_event_loop()
for word, phonemes in tested_words.items():
    task = synth.synth_ssml(ssml_template.format(word=word, phonemes=phonemes.replace(" ", "")))
    audio_output = loop.run_until_complete(task)
    with open(OUTPUT_DIR / Path(f"{word}.ogg"), "wb") as audio_file:
        audio_file.write(audio_output)
